In [1]:
import tensorflow as tf
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from collections import Counter
import collections
import random
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import Bio
from Bio import SeqIO
import os
import concurrent.futures
import functools
from functools import partial
import math
import threading
import time
import random
from random import shuffle
import pickle
import tempfile
import ntpath
import os.path

# k-mer size to use
k = 9

#
# NOTE!!!!!!!!!!!!!!!!
#
# We can reduce problem space if we get the reverse complement, and add a bit to indicate reversed or not...
# Not really.... revcomp just doubles it back up again....
#
# Also -- Build a recurrent network to predict sequences that come after a given kmer?
# Look at word2vec, dna2vec, bag of words, skip-gram
#

# Problem space
space = 5 ** k

def partition(n, step, coll):
    for i in range(0, len(coll), step):
        if (i+n > len(coll)):
            break #  raise StopIteration...
        yield coll[i:i+n]
        
def get_kmers(k):
    return lambda sequence: partition(k, k, sequence)

def convert_nt(c):
    return {"N": 0, "A": 1, "C": 2, "T": 3, "G": 4}.get(c, 0)

def convert_nt_complement(c):
    return {"N": 0, "A": 3, "C": 4, "T": 1, "G": 2}.get(c, 0)

def convert_kmer_to_int(kmer):
    return int(''.join(str(x) for x in (map(convert_nt, kmer))), 5)

def convert_kmer_to_int_complement(kmer):
    return int(''.join(str(x) for x in reversed(list(map(convert_nt_complement, kmer)))), 5)

def convert_base5(n):
    return {"0": "N", "1": "A", "2": "C", "3": "T", "4": "G"}.get(n,"N")

def convert_to_kmer(kmer):
    return ''.join(map(convert_base5, str(np.base_repr(kmer, 5))))

# Not using sparse tensors anymore.
   
tf.logging.set_verbosity(tf.logging.INFO)

# Get all kmers, in order, with a sliding window of k (but sliding 1bp for each iteration up to k)
# Also get RC for all....

def kmer_processor(seq,offset):
    return list(map(convert_kmer_to_int, get_kmers(k)(seq[offset:])))

def get_kmers_from_seq(sequence):
    kmers_from_seq = list()

    kp = functools.partial(kmer_processor, sequence)
    
    for i in map(kp, range(0,k)):
        kmers_from_seq.append(i)

    rev = sequence[::-1]
    kpr = functools.partial(kmer_processor, rev)
    
    for i in map(kpr, range(0,k)):
        kmers_from_seq.append(i)
            
#    for i in range(0,k):
#        kmers_from_seq.append(kmer_processor(sequence,i))
#    for i in range(0,k):
#        kmers_from_seq.append(kmer_processor(rev, i))
    return kmers_from_seq

data = list()

def load_fasta(filename):
    data = dict()
    file_base_name = ntpath.basename(filename)
    picklefilename = file_base_name + ".picklepickle"
    if os.path.isfile(picklefilename):
        print("Loading from pickle")
        data = pickle.load(open(picklefilename, "rb"))
    else:
        print("File not found, generating new sequence: " + picklefilename)
        for seq_record in SeqIO.parse(filename, "fasta"):
            data.update({seq_record.id:
                         get_kmers_from_seq(seq_record.seq.upper())})
        pickle.dump(data, open(picklefilename, "wb"))
    return(data)
        
def get_kmers_from_file(filename):
    kmer_list = list()
    for seq_record in SeqIO.parse(filename, "fasta"):
        kmer_list.extend(get_kmers_from_seq(seq_record.seq.upper()))
    return set([item for sublist in kmer_list for item in sublist])

all_kmers = set()

# Very slow, should make this part concurrent...

def find_all_kmers(directory):
    kmer_master_list = list()
    files = [directory + "/" + f for f in os.listdir(directory)]
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        for i in executor.map(get_kmers_from_file, files):
            kmer_master_list.extend(list(i))
            kmer_master_list = list(set(kmer_master_list))
            print("Total unique kmers: " + str(len(set(kmer_master_list))))
    return set(kmer_master_list)

def get_categories(directory):
    data = list()
    files = os.listdir(directory)
    for filename in files:
        for seq_record in SeqIO.parse(directory + "/" + filename, "fasta"):
            data.append(seq_record.id)
    data = sorted(list(set(data)))
    return(data)

In [6]:
filegen = training_file_generator("training-files/")
training_data = load_fasta(filegen())


File not found, generating new sequence: KH46c.final.fasta.picklepickle


In [2]:
replicons_list = get_categories("training-files/")


In [10]:
# Because this was run at work on a smaller sample of files....
# with open("all_kmers_subset.txt", "w") as f:
#     for s in all_kmers:
#         f.write(str(s) +"\n")

# Because this was run at work on a smaller sample of files....
all_kmers = list()
# with open("all_kmers_subset.txt", "r") as f:
#     for line in f:
#         all_kmers.append(int(line.strip()))

all_kmers = pickle.load(open("all_kmers.p", "rb"))

all_kmers = set(all_kmers)
len(all_kmers)

269132

In [11]:
# len(data)

# all_kmers = set([item for sublist in data for item in sublist])
unused_kmers = set(range(0, space)) - all_kmers

kmer_dict = dict()
reverse_kmer_dict = dict();

a = 0
for i in all_kmers:
    kmer_dict[i] = a
    reverse_kmer_dict[a] = i
    a += 1
    
kmer_count = len(all_kmers)

[len(all_kmers), len(unused_kmers), space]

[269132, 1683993, 1953125]

In [12]:
def training_file_generator(directory):
    files = [directory + "/" + f for f in os.listdir(directory)]
    random.shuffle(files)
    def gen():
        nonlocal files
        if (len(files) == 0):
            files = [directory + "/" + f for f in os.listdir(directory)]
            random.shuffle(files)
        return(files.pop())
    return gen

def gen_random_training_data(input_data, window_size):
    rname = random.choice(list(input_data.keys()))
    rdata = random.choice(input_data[rname])
    idx = random.randrange(window_size + 1, len(rdata) - window_size - 1)
    tdata = list();
    for i in range(idx - window_size - 1, idx + window_size):
        if (i < 0): continue
        if (i >= len(rdata)): break
        if type(rdata[idx]) == list: break;
        if type(rdata[i]) == list: break
        tdata.append(kmer_dict[rdata[i]])
    return tdata, rname

# The current state is, each training batch is from a single FASTA file (strain, usually)
# This can be ok, as long as training batch is a large number
# Need to speed up reading of FASTA files though, maybe pyfaidx or something?

# Define the one-hot dictionary...

oh = dict()
a = 0
for i in replicons_list:
    oh[i] = tf.one_hot(a, len(replicons_list))
    a += 1
    
oh = dict()
a = 0
for i in replicons_list:
    oh[i] = a
    a += 1
    
oh = dict()
oh['Main'] = [1.0, 0.0, 0.0]
oh['pSymA'] = [0.0, 1.0, 0.0]
oh['pSymB'] = [0.0, 0.0, 1.0]


def generate_training_batch(data, batch_size, window_size):
    training_batch_data = list();
    while len(training_batch_data) < batch_size:
         training_batch_data.append(gen_random_training_data(data, 
                                                             window_size))
    return training_batch_data

def train_input_fn():
    rdata = generate_training_batch(training_data, 1, window_size)[0]
    return rdata[0], oh[rdata[1]]
    # return {"train_input": rdata[0]}, oh[rdata[1]]
        

In [9]:
# filegen = training_file_generator("training-files/")

# training_data = load_fasta(filegen())
# training_data_backup = training_data

# len(training_data)

# gen_random_training_data(training_data, 7)

# generate_training_batch(training_data, 5, 7)

# random.choice(list(input_data.keys()))

train_input_fn()

([247872,
  221046,
  29443,
  92155,
  190386,
  173423,
  226297,
  234312,
  22989,
  217994,
  25037,
  29510,
  3023,
  223617,
  240546],
 [0.0, 0.0, 1.0])

In [10]:
valid_examples[0]

[105761,
 49545,
 256008,
 139576,
 51403,
 244727,
 259373,
 264896,
 262923,
 16082,
 261828,
 140172,
 49599,
 30035,
 29680]

In [13]:
filegen = training_file_generator("training-files/")

# training_data = load_fasta(filegen())

batch_size = 1024
embedding_size = 128
window_size = 7

validation_set = generate_training_batch(training_data, 10000, window_size)
# validation_kmers = list(set([i[0] for i in validation_set]))
# del validation_set

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 1024
valid_examples = [i[0] for i in validation_set]
num_sampled = 256
del validation_set

learning_rate = 0.1

# Network Parameters
n_hidden_1 = 500
n_hidden_2 = 50



<tf.Tensor 'ArgMax:0' shape=(3,) dtype=int64>

In [29]:
# https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/neural_network.py

<tf.Tensor 'Softmax:0' shape=(1, 3) dtype=float32>

In [15]:
graph = tf.Graph()

batch_size = 1

with graph.as_default():
    # Load embedding
    kmers = tf.Variable(tf.constant(0.0, shape=[kmer_count, 128]),
                       trainable=False, name="kmers")

    # embeddings = np.load("embeddings_200000.npy")

    embedding_placeholder = tf.placeholder(tf.int32, [kmer_count, 128])
    embedding_init = kmers.assign(embeddings)

    # Input data.
    # Take 1 kmer and the 7 on each side of it
    # So for k=9, we are testing 135bp
    # So n = 15
    train_input = tf.placeholder(tf.int32, shape=[batch_size, 15]) 
    train_label = tf.placeholder(tf.int32, shape=[batch_size, 3])
#    train_label_r = tf.reshape(train_label, [-1])
#    labels = tf.one_hot(train_label, len(replicons_list), dtype=tf.int32)

 
    kmer_input = tf.nn.embedding_lookup(embeddings, train_input)
    kmer_input_r = tf.reshape(kmer_input, [batch_size, -1]) # Flatten

    # valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # replicons = tf.placeholder(tf.int16, shape=[None, 1])

    l1 = tf.layers.dense(kmer_input_r, n_hidden_1)
    l2 = tf.layers.dense(l1, n_hidden_2)
    logits = tf.layers.dense(l2, len(replicons_list))
    
    pred_classes = tf.argmax(logits, axis=1)
    pred_prob = tf.nn.softmax(logits)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = train_label))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    
    acc = tf.metrics.accuracy(labels = tf.argmax(train_label, 1), predictions=pred_classes)
    
    init = tf.global_variables_initializer()

In [22]:
train_input_fn()

([239238,
  169260,
  242754,
  215856,
  60843,
  132808,
  249405,
  129105,
  112336,
  19860,
  139171,
  66831,
  213222,
  255357,
  21515],
 [0.0, 0.0, 1.0])

In [23]:
with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as session:
    init.run()
    num_steps = 5000
    for step in xrange(num_steps):
      myt, label = train_input_fn()
      train.run(feed_dict={train_input: [myt], 
                           train_label: [label]})
    
#    print(session.run([acc], feed_dict={train_input: [myt], train_label: [label]}))
    
    for step in xrange(10):
        myt, label = train_input_fn()
        classification = session.run([pred_classes, pred_prob], feed_dict={train_input: [myt]})
        print(classification, " ", label)
    
    
    

[array([2], dtype=int64), array([[ 0.1505875 ,  0.2130429 ,  0.63636953]], dtype=float32)]   [0.0, 1.0, 0.0]
[array([2], dtype=int64), array([[ 0.17296444,  0.41227347,  0.41476214]], dtype=float32)]   [0.0, 1.0, 0.0]
[array([2], dtype=int64), array([[ 0.20260364,  0.31371108,  0.48368534]], dtype=float32)]   [1.0, 0.0, 0.0]
[array([2], dtype=int64), array([[ 0.33472094,  0.28635615,  0.37892291]], dtype=float32)]   [1.0, 0.0, 0.0]
[array([2], dtype=int64), array([[ 0.10926115,  0.29860041,  0.59213847]], dtype=float32)]   [0.0, 0.0, 1.0]
[array([2], dtype=int64), array([[ 0.21947457,  0.25672138,  0.52380401]], dtype=float32)]   [0.0, 1.0, 0.0]
[array([2], dtype=int64), array([[ 0.1778408 ,  0.30563816,  0.51652104]], dtype=float32)]   [1.0, 0.0, 0.0]
[array([2], dtype=int64), array([[ 0.11665278,  0.31180292,  0.57154435]], dtype=float32)]   [1.0, 0.0, 0.0]
[array([2], dtype=int64), array([[ 0.16575523,  0.3371191 ,  0.49712569]], dtype=float32)]   [1.0, 0.0, 0.0]
[array([2], dtype=i

{'train_input': <tf.Tensor 'embedding_lookup_12:0' shape=(15, 128) dtype=float32>}

In [ ]:
num_steps = 5

executor = concurrent.futures.ThreadPoolExecutor(max_workers=2)
future = executor.submit(load_fasta, filegen())

tdata = list()
tdata = future.result()
print("tdata length: ", str(len(tdata)))

with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as session:
  # We must initialize all variables before we use them.
  init.run()
  print('Initialized')

  average_loss = 0
  for step in xrange(num_steps):
    
    if step % 15000 == 0: # Change files every 15k steps
        print("Loading new file at step: ", step)
        # Start loading the next file, so it has time to finish while the neural net does its training
        tdata = future.result()
        future = executor.submit(load_fasta, filegen())
        
    if step == 5:
        print("Reached step 5!")
        
    if len(tdata) == 0:
        print("Using short-circuit load-fasta at step: ", step)
        tdata = load_fasta(filegen()) # Emergency short-circuit here....
        
    batch_data = generate_training_batch(tdata, batch_size, window_size)
    feed_dict = {train_inputs: [x[0] for x in batch_data], 
                 train_labels: [[x[1]] for x in batch_data]}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    # Print status every 10k steps
    if step % 10000 == 0:
        if step > 0:
            average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
        print('Average loss at step ', step, ': ', average_loss)
        average_loss = 0
    
    # Save every 50k steps
#    if step % 100000 == 0:
#        print("Saving model at step: ", step)
#        saver.save(session, './replicon-model', global_step=step)
#        print("Saved model at step: ", step)

        
#    if step % 20000 == 0:
#        sim = similarity.eval()
#        accuracy = 0
#        for i in range(0, 100):
#            rand_kmer = random.choice(list(validation_dict.keys()))
#            top_k = 10
#            nearest = (-sim[rand_kmer, :]).argsort()[1:top_k + 1]
            


In [19]:
kmer_dict

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 11: 9,
 12: 10,
 13: 11,
 14: 12,
 16: 13,
 17: 14,
 18: 15,
 19: 16,
 21: 17,
 22: 18,
 23: 19,
 24: 20,
 31: 21,
 32: 22,
 33: 23,
 34: 24,
 36: 25,
 37: 26,
 38: 27,
 39: 28,
 41: 29,
 42: 30,
 43: 31,
 44: 32,
 46: 33,
 47: 34,
 48: 35,
 49: 36,
 56: 37,
 57: 38,
 58: 39,
 59: 40,
 61: 41,
 62: 42,
 63: 43,
 64: 44,
 66: 45,
 67: 46,
 68: 47,
 69: 48,
 71: 49,
 72: 50,
 73: 51,
 74: 52,
 81: 53,
 82: 54,
 83: 55,
 84: 56,
 86: 57,
 87: 58,
 88: 59,
 89: 60,
 91: 61,
 92: 62,
 93: 63,
 94: 64,
 96: 65,
 97: 66,
 98: 67,
 99: 68,
 106: 69,
 107: 70,
 108: 71,
 109: 72,
 111: 73,
 112: 74,
 113: 75,
 114: 76,
 116: 77,
 117: 78,
 118: 79,
 119: 80,
 121: 81,
 122: 82,
 123: 83,
 124: 84,
 156: 85,
 158: 86,
 159: 87,
 161: 88,
 162: 89,
 163: 90,
 164: 91,
 166: 92,
 168: 93,
 169: 94,
 171: 95,
 172: 96,
 173: 97,
 174: 98,
 181: 99,
 182: 100,
 183: 101,
 184: 102,
 186: 103,
 187: 104,
 188: 105,
 189: 106,
 191: 107,


In [70]:
def model_fn(features, labels, mode):
    kmers = tf.Variable(tf.constant(0.0, shape=[kmer_count, 128]),
                        trainable=False, name="kmers")
    
    embedding_placeholder = tf.placeholder(tf.int32, [kmer_count, 128])
    embedding_init = kmers.assign(embeddings)
    
    train_input = tf.placeholder(tf.int32, shape=[batch_size, 15]) 
    train_label = tf.placeholder(tf.int32, shape=[batch_size, 3])
    
    kmer_input = tf.nn.embedding_lookup(embeddings, train_input)
    kmer_input_r = tf.reshape(kmer_input, [batch_size, -1])
    
    l1 = tf.layers.dense(kmer_input_r, n_hidden_1)
    l2 = tf.layers.dense(l1, n_hidden_2)
    logits = tf.layers.dense(l2, len(replicons_list))
    
    pred_classes = tf.argmax(logits, axis=1)
    pred_prob = tf.nn.softmax(logits)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = train_label))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train = optimizer.minimize(loss, global_step=tf.train.get_global_step())

    acc = tf.metrics.accuracy(labels = tf.argmax(train_label, 1), predictions=pred_classes)

    estim_specs = tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=pred_classes,
            loss=loss,
            train_op=train,
            eval_metric_ops={'accuracy': acc})

    return estim_specs

In [29]:
tbatch = generate_training_batch(training_data, 1, window_size)
len(tbatch[0][0])

15

In [100]:
embeddings = np.load("embeddings_200000.npy")

def get_kmer_embeddings(kmers):
    a = list() # numpy.empty(128 * 15)
    for k in kmers:
        a.append(embeddings[k])
    return np.hstack(a)

# training_data = load_fasta(filegen())
get_kmer_embeddings(tbatch[0][0])

tf.convert_to_tensor([tf.convert_to_tensor(get_kmer_embeddings(tbatch[0][0]))])

<tf.Tensor 'packed:0' shape=(1, 1920) dtype=float32>

In [111]:
repdict = dict()
a = 0
for i in replicons_list:
    repdict[i] = a
    a += 1

def train_input_fn(data):
    tbatch = generate_training_batch(data, 1, window_size)
    dat = {'x': tf.convert_to_tensor([tf.convert_to_tensor(get_kmer_embeddings(tbatch[0][0]))])}
    lab = tf.convert_to_tensor([repdict[tbatch[0][1]]])
    return dat, lab

def test_input_fn(data):
    tbatch = generate_training_batch(data, 1, window_size)
    dat = {'x': tf.convert_to_tensor([tf.convert_to_tensor(get_kmer_embeddings(tbatch[0][0]))])}
    lab = tf.convert_to_tensor([repdict[tbatch[0][1]]])
    return dat, lab

In [112]:
feature_columns = [tf.feature_column.numeric_column("x", shape=[1920])]
nn = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                hidden_units = [5000,500,50],
                                activation_fn=tf.nn.relu,
                                dropout=0.2,
                                n_classes=len(replicons_list),
                                optimizer="Adam")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9xj4sya3', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [113]:
for i in xrange(10):
    training_data = load_fasta(filegen())
    tfn = functools.partial(train_input_fn, training_data)
    nn.train(input_fn=tfn, steps=1000)

File not found, generating new sequence: Rm1021.final.fasta.picklepickle
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp9xj4sya3/model.ckpt.
INFO:tensorflow:loss = 3.34785, step = 1
INFO:tensorflow:global_step/sec: 28.5192
INFO:tensorflow:loss = 0.0, step = 101 (3.507 sec)
INFO:tensorflow:global_step/sec: 28.1805
INFO:tensorflow:loss = 0.0, step = 201 (3.548 sec)
INFO:tensorflow:global_step/sec: 28.4364
INFO:tensorflow:loss = 0.0, step = 301 (3.516 sec)
INFO:tensorflow:global_step/sec: 28.1645
INFO:tensorflow:loss = 0.0, step = 401 (3.551 sec)
INFO:tensorflow:global_step/sec: 28.2323
INFO:tensorflow:loss = 0.0, step = 501 (3.542 sec)
INFO:tensorflow:global_step/sec: 28.1659
INFO:tensorflow:loss = 0.0, step = 601 (3.551 sec)
INFO:tensorflow:global_step/sec: 28.8472
INFO:tensorflow:loss = 0.0, step = 701 (3.466 sec)
INFO:tensorflow:global_step/sec: 28.4099
INFO:tensorflow:loss = 0.0, step = 801 (3.520 sec)
INFO:tensorflow:global_step/sec

In [138]:
training_data = load_fasta(filegen())
tfn = functools.partial(train_input_fn, training_data)
accuracy_score = nn.evaluate(input_fn=tfn, steps=10)['accuracy']
print("\nTest Accuracy: {0:f}\n".format(accuracy_score))


File not found, generating new sequence: USDA1106.final.fasta.picklepickle
INFO:tensorflow:Starting evaluation at 2017-10-20-22:16:29
INFO:tensorflow:Restoring parameters from /tmp/tmp9xj4sya3/model.ckpt-10000
INFO:tensorflow:Evaluation [1/10]
INFO:tensorflow:Evaluation [2/10]
INFO:tensorflow:Evaluation [3/10]
INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2017-10-20-22:16:29
INFO:tensorflow:Saving dict for global step 10000: accuracy = 1.0, average_loss = 0.0, global_step = 10000, loss = 0.0

Test Accuracy: 1.000000



In [135]:
tfn = functools.partial(train_input_fn, training_data)

TypeError: run() missing 1 required positional argument: 'fetches'